In [125]:
import ccxt
import os
import datetime
import math

from dotenv import dotenv_values, load_dotenv
from pprint import pprint
from arch import arch_model

# import ccxt.async_support as ccxt # link against the asynchronous version of ccxt

from functions.BS_pricer import BS_pricer
from functions.Parameters import Option_param
from functions.Processes import Diffusion_process

import numpy as np
import pandas as pd
import scipy as scp
import scipy.stats as ss
from scipy.integrate import quad
from functools import partial

import matplotlib.pyplot as plt
%matplotlib inline

In [126]:
# Load environment variables from .env file
load_dotenv()

# Set the API keys for the exchanges
delta_api_key = os.getenv('DELTA_API_KEY')
delta_api_secret = os.getenv('DELTA_API_SECRET')
deribit_api_key = os.getenv('DERIBIT_API_KEY')
deribit_api_secret = os.getenv('DERIBIT_API_SECRET')

# Create the exchange objects
delta = ccxt.delta({
    'apiKey': delta_api_key,
    'secret': delta_api_secret,
})
deribit = ccxt.deribit({
    'apiKey': deribit_api_key,
    'secret': deribit_api_secret,
})

# Create the dictionary
exchanges_dict = {'delta': delta, 'deribit': deribit}
pprint(exchanges_dict)

{'delta': ccxt.delta(), 'deribit': ccxt.deribit()}


In [127]:
# delta_instruments = exchanges_dict['delta'].fetchMarkets()
# deribit_instruments = exchanges_dict['deribit'].fetchMarkets()
instruments_dict = {k: exchanges_dict[k].fetchMarkets() for k in exchanges_dict}
instruments_dict['delta'] = {d['symbol']: d for d in instruments_dict['delta']}
instruments_dict['deribit'] = {d['symbol']: d for d in instruments_dict['deribit']}

/tmp/ipykernel_68928/3934710538.py:3: RuntimeWarning: coroutine 'delta.fetch_markets' was never awaited
  instruments_dict = {k: exchanges_dict[k].fetchMarkets() for k in exchanges_dict}
/tmp/ipykernel_68928/3934710538.py:3: RuntimeWarning: coroutine 'deribit.fetch_markets' was never awaited
  instruments_dict = {k: exchanges_dict[k].fetchMarkets() for k in exchanges_dict}


In [128]:
# We define the contract we are looking for
contract_specification = {
    'expiration': '230127',
    'payoff': 'C',
    'underlying': 'BTC',
    'strike': 17000,
    'quote': 'USDT'
}

In [129]:
def get_ticker(strike, expiration, underlying, quote, payoff):
    # Construct the ticker symbol for the contract
    delta_ticker_specification = f"{underlying}/{quote}:{quote}-{expiration}:{strike}:{payoff}"
    if quote == "USDT": quote = "USD"
    deribit_ticker_specification = f"{underlying}/{quote}:{underlying}-{expiration}-{strike}-{payoff}"
    # Fetch the ticker data from the Delta exchange
    delta_ticker = exchanges_dict['delta'].fetch_ticker(delta_ticker_specification)
    pprint(f"Delta ticker: {delta_ticker}")

    # Fetch the ticker data from the Deribit exchange
    deribit_ticker = exchanges_dict['deribit'].fetch_ticker(deribit_ticker_specification)
    pprint(f"Deribit ticker: {deribit_ticker}")

    return {
        "delta": delta_ticker,
        "deribit": deribit_ticker,
    }
    
tickers_dict = get_ticker(**contract_specification)

("Delta ticker: {'symbol': 'BTC/USDT:USDT-230127:17000:C', 'timestamp': "
 "1673179206884, 'datetime': '2023-01-08T12:00:06.884Z', 'high': 467.6, 'low': "
 "433.1, 'bid': None, 'bidVolume': None, 'ask': None, 'askVolume': None, "
 "'vwap': 16934.857787111112, 'open': 449.1, 'close': 433.1, 'last': 433.1, "
 "'previousClose': None, 'change': -16.0, 'percentage': -3.562680917390336, "
 "'average': 441.1, 'baseVolume': 0.27, 'quoteVolume': 4572.41160252, 'info': "
 "{'close': '433.1', 'contract_type': 'call_options', 'greeks': {'delta': "
 "'0.49056019', 'gamma': '0.00032807', 'rho': '4.08807636', 'spot': "
 "'16927.422068700555', 'theta': '-12.75938381', 'vega': '15.40314457'}, "
 "'high': '467.6', 'low': '433.1', 'mark_price': '450.36159851', 'mark_vol': "
 "'0.3148419250889370838801066081', 'oi': '4.3760', 'oi_contracts': '4376', "
 "'oi_value': '4.3760', 'oi_value_symbol': 'BTC', 'oi_value_usd': "
 "'74095.3660', 'open': '449.1', 'price_band': {'lower_limit': "
 "'0.100000000000000000

In [130]:
def parse_option_spectification_delta(option_specification):
    expiration = datetime.datetime.strptime(option_specification['expiration'], "%y%m%d")

    return {
        "underlying": option_specification['underlying'],
        "quote": option_specification['quote'],
        "expiration": expiration,
        "strike": float(option_specification['strike']),
        "payoff": "call" if option_specification['payoff'] in ("C", "call", "Call", "CALL") else "put",
    }

option_specification = parse_option_spectification_delta(contract_specification)
pprint(option_specification)

{'expiration': datetime.datetime(2023, 1, 27, 0, 0),
 'payoff': 'call',
 'quote': 'USDT',
 'strike': 17000.0,
 'underlying': 'BTC'}


In [131]:
now = datetime.datetime.now()
forward_date = option_specification["expiration"]
print((now, forward_date))

timedelta_until_expiration = forward_date - now

days_until_expiration = timedelta_until_expiration.total_seconds() / 86400
print(days_until_expiration)

exchanges_option_contract = {}

(datetime.datetime(2023, 1, 8, 13, 0, 8, 572534), datetime.datetime(2023, 1, 27, 0, 0))
18.458234114189814


In [132]:
pprint(tickers_dict['delta']['symbol'])

# pprint(instruments_dict['delta'][tickers_dict['delta']['symbol']])
delta_maker_fees_rate = float(instruments_dict['delta'][tickers_dict['delta']['symbol']]['maker'])*100
delta_taker_fees_rate = float(instruments_dict['delta'][tickers_dict['delta']['symbol']]['taker'])*100
delta_premium_commission_rate = float(instruments_dict['delta'][tickers_dict['delta']['symbol']]['info']['product_specs']['premium_commission_rate'])

pprint([
    delta_maker_fees_rate,
    delta_taker_fees_rate,
    delta_premium_commission_rate
])

'BTC/USDT:USDT-230127:17000:C'
[0.03, 0.03, 0.1]


In [133]:
pprint(tickers_dict['deribit']['symbol'])
# pprint(instruments_dict['deribit'][tickers_dict['deribit']['symbol']])

deribit_maker_fees_rate = float(instruments_dict['deribit'][tickers_dict['deribit']['symbol']]['maker'])*100
deribit_taker_fees_rate = float(instruments_dict['deribit'][tickers_dict['deribit']['symbol']]['taker'])*100
deribit_premium_commission_rate = 0.125

pprint([
    deribit_maker_fees_rate,
    deribit_taker_fees_rate,
    deribit_premium_commission_rate
])

'BTC/USD:BTC-230127-17000-C'
[0.03, 0.03, 0.125]


In [134]:
"""
try:
    a = exchanges_dict['delta'].fetchFees()
    pprint(f"fetchFees: {a}")
    #b = exchanges_dict['delta'].fetchTradingFee(tickers_dict['delta']['info']['product_id'])
    #pprint(f"fetchFees: {b}")
    #c = exchanges_dict['delta'].fetchTradingFees()
    #pprint(f"fetchFees: {c}")
    # d = exchanges_dict['delta'].fetchTransactionFee()
    # pprint(f"fetchFees: {d}")
    #e = exchanges_dict['delta'].fetchTransactionFees()
    #pprint(f"fetchFees: {e}")
    #a = exchanges_dict['deribit'].fetchFees()
    #pprint(f"fetchFees: {a}")
    b = exchanges_dict['deribit'].fetchTradingFee(tickers_dict['deribit']['info']['instrument_name'])
    pprint(f"fetchFees: {b}")
    c = exchanges_dict['deribit'].fetchTradingFees()
    pprint(f"fetchFees: {c}")
    d = exchanges_dict['deribit'].fetchTransactionFee()
    pprint(f"fetchFees: {d}")
    e = exchanges_dict['deribit'].fetchTransactionFees()
    pprint(f"fetchFees: {e}")
except Exception as e:
    pprint(e)
"""

'\ntry:\n    a = exchanges_dict[\'delta\'].fetchFees()\n    pprint(f"fetchFees: {a}")\n    #b = exchanges_dict[\'delta\'].fetchTradingFee(tickers_dict[\'delta\'][\'info\'][\'product_id\'])\n    #pprint(f"fetchFees: {b}")\n    #c = exchanges_dict[\'delta\'].fetchTradingFees()\n    #pprint(f"fetchFees: {c}")\n    # d = exchanges_dict[\'delta\'].fetchTransactionFee()\n    # pprint(f"fetchFees: {d}")\n    #e = exchanges_dict[\'delta\'].fetchTransactionFees()\n    #pprint(f"fetchFees: {e}")\n    #a = exchanges_dict[\'deribit\'].fetchFees()\n    #pprint(f"fetchFees: {a}")\n    b = exchanges_dict[\'deribit\'].fetchTradingFee(tickers_dict[\'deribit\'][\'info\'][\'instrument_name\'])\n    pprint(f"fetchFees: {b}")\n    c = exchanges_dict[\'deribit\'].fetchTradingFees()\n    pprint(f"fetchFees: {c}")\n    d = exchanges_dict[\'deribit\'].fetchTransactionFee()\n    pprint(f"fetchFees: {d}")\n    e = exchanges_dict[\'deribit\'].fetchTransactionFees()\n    pprint(f"fetchFees: {e}")\nexcept Exception

In [135]:
### DeltaExchange Format
"""
    Retrieve
    IV
    index price (Underlying price)
    days until expiration
    strike price
    interest rate
    dividend yield (0)
"""
delta_option_contract = tickers_dict['delta']
greeks = delta_option_contract["info"]["greeks"]
ask_iv = float(delta_option_contract["info"]["quotes"]['ask_iv'])
ask_size = float(delta_option_contract["info"]["quotes"]['ask_size'])
best_ask = float(delta_option_contract["info"]["quotes"]['best_ask'])
best_bid = float(delta_option_contract["info"]["quotes"]['best_bid'])
bid_iv = float(delta_option_contract["info"]["quotes"]['bid_iv'])
bid_size = float(delta_option_contract["info"]["quotes"]['bid_size'])
mark_iv = float(delta_option_contract["info"]["quotes"]['mark_iv'])
strike_price = float(delta_option_contract["info"]["strike_price"])
spot_price = float(delta_option_contract["info"]["spot_price"])

exchanges_option_contract.update(
    {
        'delta': {
            'best_ask': best_ask,
            'spot_price': spot_price,
            'ask_iv': ask_iv,
        }
    }
)

In [136]:
### Deribit Format
deribit_option_contract = tickers_dict['deribit']
greeks = deribit_option_contract["info"]["greeks"] # Deribit format
ask_iv = float(deribit_option_contract['info']['ask_iv']) if float(deribit_option_contract['info']['ask_iv']) else float(deribit_option_contract['info']['mark_iv']) 
strike_price = option_specification['strike']
spot_price = float(deribit_option_contract["info"]["underlying_price"])
best_ask_price = float(deribit_option_contract["info"]['best_ask_price']) if float(deribit_option_contract['info']['best_ask_price']) else float(deribit_option_contract['info']['mark_price']) 

exchanges_option_contract.update(
    {
        'deribit': {
            'best_ask': best_ask_price*spot_price,
            'spot_price': spot_price,
            'ask_iv': ask_iv/100.0,
        }
    }
)

In [137]:
def binomial_price(S0, K, T, r, sigma, N=15000, payoff="call"):
    # N number of periods or number of time steps  

    dT = float(T) / N                             # Delta t
    u = np.exp(sigma * np.sqrt(dT))                 # up factor
    d = 1.0 / u                                   # down factor 

    V = np.zeros(N+1)                             # initialize the price vector
    S_T = np.array( [(S0 * u**j * d**(N - j)) for j in range(N + 1)] )  # price S_T at time T

    a = np.exp(r * dT)    # risk free compounded return
    p = (a - d)/ (u - d)  # risk neutral up probability
    q = 1.0 - p           # risk neutral down probability   

    if payoff =="call":
        V[:] = np.maximum(S_T-K, 0.0)
    else:
        V[:] = np.maximum(K-S_T, 0.0)

    for i in range(N-1, -1, -1):
        V[:-1] = np.exp(-r*dT) * (p * V[1:] + q * V[:-1])    # the price vector is overwritten at each step
            
    return V[0]


In [138]:
for exch, option  in exchanges_option_contract.items():
   print("==============================================")
   info = {
      "S0": float(option['spot_price']),
      "K": float(option_specification['strike']),
      "T": float(days_until_expiration/365),
      "r": float(1/100),
      "sigma": float(option['ask_iv']),
      "payoff": option_specification["payoff"]
   }
   option_price = BS_pricer.BlackScholes(**info)

   print(f"Our BS price: {option_price}")
   #bin_price = binomial_price(**info)
   #print(f"binomial price: {bin_price}") 
   print(f"current market price on {exch} = {option['best_ask']}")
print("==============================================")

Our BS price: 467.48739214286707
current market price on delta = 471.0
Our BS price: 456.61298329627334
current market price on deribit = 457.41833999999994


In [139]:
header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
# since = exchange.milliseconds () - 86400000  # -1 day from now
start_date = '2023-01-01T00:00:00Z' # fetch from a certain starting datetime
t_frame = '1d' # 1-day timeframe, usually from 1-minute to 1-week depending on the exchange
symbols = [
    'BTC/USDT:USDT',
    'ETH/USDT:USDT',
    #'AAVE/USDT:USDT',
    #'BNB/USDT:USDT',
    #'CRV/USDT:USDT',
    #'SNX/USDT:USDT',
    #'LINK/USDT:USDT',
]
exchange_list = ['delta']#, 'deribit']
data_df = pd.DataFrame(None, columns=header).set_index('Timestamp')

for exchange_name, exch in exchanges_dict.items():
    print(exch)
    if exch.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print(f'{exchange_name} does not support fetching OHLC data. Please use another exchange')
        print('-'*80)
        quit()
    if (not hasattr(exch, 'timeframes')) or (t_frame not in exch.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print(f'The requested timeframe ({t_frame}) is not available from {exchange_name}\n')
        print('Available timeframes are:')
        for key in exch.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exch.load_markets()
    for coin in symbols:
        print(coin)
        since = exch.parse8601(start_date) # fetch from a certain starting datetime
        all_ohlcv = []
        while since < exch.milliseconds():
            coin_ohlcv = exch.fetchOHLCV(coin, t_frame, since)
            if not coin_ohlcv: break
            since = coin_ohlcv[-1][0] + 1 # query the last and add 1 to the number of milliseconds
            all_ohlcv += coin_ohlcv
        df = pd.DataFrame(all_ohlcv, columns=header).set_index('Timestamp')
        df['Symbol'] = coin
        df['Exchange'] = exch
        data_df = pd.concat([data_df, df])

data_df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
data_df['Date'] = pd.to_datetime(data_df.index, unit='s')
data_df.to_csv(f"./data/{start_date}_{t_frame}")

Delta Exchange
BTC/USDT:USDT
ETH/USDT:USDT
Deribit
BTC/USDT:USDT


BadSymbol: deribit does not have market symbol BTC/USDT:USDT

In [ ]:
### DATA RETRIEVE BACKUP

for exch in exchange_list:
    print(exch)
    try:
        exchange = getattr(ccxt, exch)()
    except AttributeError:
        print('-'*36,' ERROR ','-'*35)
        print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
        print('-'*80)
        quit()
    if exchange.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print('{} does not support fetching OHLC data. Please use another exchange'.format(exch))
        print('-'*80)
        quit()
    if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
        print('Available timeframes are:')
        for key in exchange.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exchange.load_markets()
    for coin in symbols:
        print(coin)
        since = exchange.parse8601(start_date) # fetch from a certain starting datetime
        all_ohlcv = []
        while since < exchange.milliseconds():
            coin_ohlcv = exchange.fetchOHLCV(coin, t_frame, since)
            if not coin_ohlcv: break
            since = coin_ohlcv[-1][0] + 1 # query the last and add 1 to the number of milliseconds
            all_ohlcv += coin_ohlcv
        df = pd.DataFrame(all_ohlcv, columns=header).set_index('Timestamp')
        df['Symbol'] = coin
        df['Exchange'] = exch
        data_df = pd.concat([data_df, df])

data_df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
data_df['Date'] = pd.to_datetime(data_df.index, unit='s')
data_df.to_csv(f"./data/{start_date}_{t_frame}")

In [21]:
df = pd.read_csv(f"./data/{start_date}_{t_frame}", index_col="Date")
df

,Timestamp,Open,High,Low,Close,Volume,Symbol,Exchange
Date,,,,,,,,
2021-01-01,1.609459e+09,28989.5,29633.5,28699.5,29339.0,543183.0,BTC/USDT:USDT,delta
2021-01-02,1.609546e+09,29311.0,33311.0,28990.0,32222.0,671497.0,BTC/USDT:USDT,delta
2021-01-03,1.609632e+09,32237.0,34776.5,32037.0,33021.0,478717.0,BTC/USDT:USDT,delta
2021-01-04,1.609718e+09,33032.5,33620.0,28322.0,32027.5,530053.0,BTC/USDT:USDT,delta
2021-01-05,1.609805e+09,32087.5,34446.5,29968.0,34040.0,670836.0,BTC/USDT:USDT,delta
...,...,...,...,...,...,...,...,...
2022-12-16,1.671149e+09,17352.0,17528.0,16519.5,16624.5,2247244.0,BTC/USDT:USDT,delta
2022-12-17,1.671235e+09,16623.5,16789.0,16575.5,16769.5,1329591.0,BTC/USDT:USDT,delta
2022-12-18,1.671322e+09,16768.5,16871.0,16655.0,16731.0,1221652.0,BTC/USDT:USDT,delta


In [22]:
returns = 100 * df[df["Symbol"] == "BTC/USDT:USDT"]["Close"].pct_change().dropna()
returns

Date
2021-01-02    9.826511
2021-01-03    2.479672
2021-01-04   -3.008691
2021-01-05    6.283662
2021-01-06    8.168331
                ...   
2022-12-16   -4.192600
2022-12-17    0.872207
2022-12-18   -0.229583
2022-12-19   -1.793079
2022-12-20    2.270099
Name: Close, Length: 718, dtype: float64

In [23]:
am = arch_model(returns, vol="Garch", p=1, o=0, q=1, dist="Normal")
res = am.fit(update_freq=5)

res = am.fit()
forecast = res.forecast(horizon=1, reindex=False)
variance_forecast = forecast.variance.iloc[-1][0]
volatility_forecast = np.sqrt(variance_forecast)
annualized_volatility_forecast = volatility_forecast * np.sqrt(252) / 100
# Cette VOL est issu de la prediction du GARCH a comparer avec les IV de l'Option Wheel 
# et trade le mispricing
annualized_volatility_forecast

Iteration:      5,   Func. Count:     35,   Neg. LLF: 1986.6276349780278
Iteration:     10,   Func. Count:     64,   Neg. LLF: 1970.9922385375985
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1970.9839043467969
            Iterations: 14
            Function evaluations: 86
            Gradient evaluations: 14
Iteration:      1,   Func. Count:      6,   Neg. LLF: 424074121.17835295
Iteration:      2,   Func. Count:     14,   Neg. LLF: 2338.7083696338486
Iteration:      3,   Func. Count:     22,   Neg. LLF: 2039.6286477783688
Iteration:      4,   Func. Count:     29,   Neg. LLF: 1986.3394479465649
Iteration:      5,   Func. Count:     35,   Neg. LLF: 1986.6276349780278
Iteration:      6,   Func. Count:     41,   Neg. LLF: 1985.7314531882776
Iteration:      7,   Func. Count:     47,   Neg. LLF: 1983.261585117878
Iteration:      8,   Func. Count:     53,   Neg. LLF: 1971.7411278357495
Iteration:      9,   Func. Count:     59,   Neg. LLF: 1970.99

0.4979188297159979

In [24]:
call_price = BS_pricer.BlackScholes("put", S0=S0, K=K, T=T ,r=r, sigma=annualized_volatility_forecast)
print(call_price) # our price

0.003873559103627855


In [ ]:
base = datetime.datetime.today()

In [ ]:
s = exchange.parse8601(base)

In [ ]:
exchange.milliseconds()

In [50]:
df['symbol'] = symbol
df['exchange'] = exch
df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
df['Date'] = pd.to_datetime(df.index, unit='s')
df.to_csv(f"./data/{exch}_{t_frame}")

In [51]:
len(all_ohlcv)

16771

In [14]:
def price_range(iv, current_price, day_to_expiration):
    range_delta = current_price*iv/100*math.sqrt(day_to_expiration/365)
    return f"[{current_price - range_delta:.02f}, {current_price + range_delta:.02f}]"

In [17]:
price_range(30, 1251, 1)

'[1231.36, 1270.64]'